In [1]:
import os, re
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import treebank
from nltk.corpus import dependency_treebank
from nltk.grammar import DependencyGrammar
from nltk.parse import DependencyGraph
import pprint
from nltk.parse.corenlp import CoreNLPDependencyParser
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
import numpy as np
import re
from numpy.linalg import norm

In [2]:
ps = PorterStemmer()
wnl = WordNetLemmatizer()
dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')

In [3]:
class Habitat:
    oid = -1
    name = ""

In [4]:
train_path = "./BioNLP-OST-2019_BB-norm_train/"
test_path = "./BioNLP-OST-2019_BB-norm_test/"
dev_path = "./BioNLP-OST-2019_BB-norm_dev/"
BB_train = os.listdir(train_path)
BB_test = os.listdir(test_path)
BB_dev = os.listdir(dev_path)
obp_file = "OntoBiotope_BioNLP-OST-2019.obo"

In [5]:
ontology = open(obp_file,encoding="utf8").read()

In [6]:
BB_train_a1 = sorted([name for name in BB_train if name.endswith("a1")])
BB_train_a2 = sorted([name for name in BB_train if name.endswith("a2")])
BB_train_txt = sorted([name for name in BB_train if name.endswith("txt")])

BB_dev_a1 = sorted([name for name in BB_dev if name.endswith("a1")])
BB_dev_a2 = sorted([name for name in BB_dev if name.endswith("a2")])
BB_dev_txt = sorted([name for name in BB_dev if name.endswith("txt")])

BB_test_a1 = sorted([name for name in BB_test if name.endswith("a1")])
BB_test_txt = sorted([name for name in BB_test if name.endswith("txt")])

In [7]:
def original_phrase(phrase):
    return phrase

In [8]:
def stem_phrase(phrase):
    #print(type(phrase))
    #print(phrase)
    
    phrase = phrase.split(" ")
    for i in range(len(phrase)):
        phrase[i] = ps.stem(phrase[i])
    phrase = " ".join(phrase)
    return phrase

In [9]:
def lemmatize_phrase(phrase):
    #print(type(phrase))
    #print(phrase)
    
    phrase = phrase.split(" ")
    for i in range(len(phrase)):
        phrase[i] = wnl.lemmatize(phrase[i])
    phrase = " ".join(phrase)
    return phrase

In [10]:
phrase_reducer = lemmatize_phrase

In [11]:
def get_headword(phrase):
    #print(phrase)
    if phrase == phrase.upper():
        return ""
    
    parses = dep_parser.parse(phrase.split())
    triples = [[(governor, dep, dependent) for governor, dep, dependent in parse.triples()] for parse in parses]
    
    try:
        if(len(triples) > 0 and len(triples[0]) > 0 and len(triples[0][0]) > 0 and len(triples[0][0][0]) > 0 and triples[0][0][0][0] is not None):
            return triples[0][0][0][0]
        else:
            return phrase
    except:
        return phrase
    
    

In [12]:
'asdsa-asdas asda-asda'.replace('-', ' ')

'asdsa asdas asda asda'

In [13]:
def add_documents(txt, processor):
    document = []
    txt = processor(txt)
    sentence_list = sent_tokenize(txt)
    
    for sentence in sentence_list:
        sentence = sentence.replace('-', ' ')
        word_list = word_tokenize(sentence)
        word_list = [word for word in word_list if word not in stopwords.words("english") and word.isalnum()]
        for i in range(len(word_list)):
            if word_list[i].isnumeric():
                word_list[i] = "##number##"
            if word_list[i] != word_list[i].upper():
                word_list[i] = processor(word_list[i].lower())
             
        document.append(word_list)
    #print(document)
    return document

In [14]:
documents = []
for i in range(len(BB_train_txt)):
    txt = open(train_path + BB_train_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)
for i in range(len(BB_dev_txt)):
    txt = open(dev_path + BB_dev_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)
for i in range(len(BB_test_txt)):
    txt = open(test_path + BB_test_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)

In [15]:
model = Word2Vec(documents, min_count=1)

In [16]:
phrase_reducer('Nares')

'Nares'

In [17]:
model['naris']

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 2.1308158e-03, -2.2164762e-03, -2.8216752e-04,  2.5654319e-03,
        1.1615772e-03,  4.5166295e-03,  4.8639855e-04, -6.9007110e-05,
       -7.8822341e-05, -1.0353950e-03, -2.3537881e-03,  4.7736559e-03,
       -4.0575503e-03, -7.7695836e-04, -9.9828828e-04,  8.0799224e-04,
        3.5232885e-03, -4.0239394e-03, -1.2739117e-03,  3.3240321e-03,
       -4.4846782e-03, -5.1341229e-03, -3.5930551e-03, -2.4225018e-03,
        2.1629422e-03, -2.9072179e-03,  9.0341031e-04, -3.0462702e-03,
       -1.1809039e-03, -1.1123555e-03,  2.0066591e-03, -2.4669820e-03,
       -3.5836536e-03, -4.7736001e-04, -1.0028803e-04,  3.8155513e-03,
       -6.7283487e-04,  2.2406841e-03, -2.0636226e-05,  3.6955872e-03,
       -2.7335507e-03, -1.1125446e-03, -1.8393438e-03,  1.9911781e-03,
       -4.7250165e-04, -2.3009779e-03, -3.8166340e-03, -3.1367920e-03,
        1.2888731e-03,  1.6501062e-03,  3.1545428e-03,  1.6542213e-03,
       -4.0891077e-03, -5.3412938e-03,  2.3378138e-03, -4.2223698e-03,
      

In [18]:
len(model.wv.vocab)

5537

In [19]:
type(list(model.wv.vocab))

list

In [20]:
len(model[list(model.wv.vocab)[0]])

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


100

In [97]:
def get_phrase_embedding(phrase, model, processor):
    original_phrase = phrase
    phrase = phrase.replace('-', ' ')
    hyphenless_phrase = phrase
    phrase = nltk.word_tokenize(phrase)
    
    phrase = [processor(word) for word in phrase if word.lower() not in stopwords.words("english") and word.isalnum()]
    for i in range(len(phrase)):
        if phrase[i].isnumeric():
            phrase[i] = "##number##"
    
    phrase_size = len(phrase)
    vec_size = len(model[list(model.wv.vocab)[0]])
    embedded_sum = np.zeros(vec_size)
    if(phrase_size == 0):
        return -1
    for word in phrase:
        if word not in list(model.wv.vocab):
            return -1
        embedded_sum = embedded_sum + np.array(model[word])
    embedded_sum = embedded_sum / phrase_size
    
    hw = get_headword(hyphenless_phrase)
    if hw == "":
        print(hyphenless_phrase)
    else:
        hw_emb = np.array(model[processor(hw)])
        embedded_sum = (4*embedded_sum + hw_emb)/5
    
    return embedded_sum
    

In [98]:
def generate_habitat_map(processor, model):
    habitat_list = ontology.split("\n\n[Term]\n")[1:]
    habitat_map = {}
    habitat_map_originals = {}
    habitat_embeddings = {}
    for h in habitat_list:
        #new_habitat = Habitat()
        h = h.split("\n")
        oid = re.findall(r"(?<=id: OBT:)[0-9]+", h[0])[0]
        name = (re.findall(r"(?<=name: ).+", h[1])[0])
        original_name = name
        if name != name.upper():
            name = name.lower()
            name = processor(name)
    
        habitat_map[name] = oid
        habitat_map_originals[oid] = name
        embedding = get_phrase_embedding(name, model, processor)
        if type(embedding) == type(np.array([1])):
            habitat_embeddings[name] = embedding
    
        # word2vec buraya eklenmeli 
        for h_line in h:
            if "synonym" in h_line:
                synonym_name = (re.findall(r"(?<=synonym: \").+(?=\")", h_line)[0])
                if synonym_name != synonym_name.upper():
                    synonym_name = synonym_name.lower()
                    synonym_name = processor(synonym_name)
                if "EXACT" in h_line and synonym_name not in habitat_map:
                    habitat_map[synonym_name] = oid
                    if type(embedding) == type(np.array([1])):
                        habitat_embeddings[synonym_name] = embedding
                    
    return habitat_map, habitat_map_originals, habitat_embeddings

In [99]:
habitat_map, habitat_map_originals, habitat_embeddings = generate_habitat_map(phrase_reducer, model)

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [100]:
len(habitat_map)

4000

In [101]:
len(habitat_embeddings)

938

In [102]:
def add_habitats(a1, a2, processor, model):
    a2 = a2.split("\n")
    a1 = a1.split("\n")
    
    
    for annotation in a2:
        a1_line = 0
        t = ""
        oid = ""
        #print(annotation)
        if "OntoBiotope" in annotation:
            t = re.findall(r"(?<=Annotation:)T[0-9]+", annotation)[0]
            oid = re.findall(r"(?<=Referent:OBT:)[0-9]+", annotation)[0]
            named_entity = ""
            
            while t not in a1[a1_line] and a1_line < len(a1)-1:
                a1_line += 1
            if "Habitat" in a1[a1_line]: 
                named_entity = (a1[a1_line].split("\t")[2])
                named_entity = named_entity.replace('-', ' ')
                if named_entity != named_entity.upper():
                    named_entity = named_entity.lower()
                    named_entity = processor(named_entity)
                if named_entity not in habitat_map:
                    habitat_map[named_entity] = oid
                
                if habitat_map_originals[oid] not in habitat_embeddings:
                    habitat_embeddings[habitat_map_originals[oid]] = []
                
                if type(habitat_embeddings[habitat_map_originals[oid]]) == list:
                    emb = get_phrase_embedding(named_entity, model, processor)
                    if type(emb) == int:
                        print("lol")
                    habitat_embeddings[habitat_map_originals[oid]].append(emb)

In [103]:
for i in range(len(BB_train_txt)):    
    a1 = open(train_path + BB_train_a1[i],encoding="utf8").read()
    a2 = open(train_path + BB_train_a2[i],encoding="utf8").read()
    add_habitats(a1, a2, phrase_reducer, model)

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


lol
lol


In [104]:
for key in habitat_embeddings:
    if type(habitat_embeddings[key]) == list:
        vec_size = len(habitat_embeddings[key][0])
        temp_vec = np.zeros(vec_size)
        for vec in habitat_embeddings[key]:
            temp_vec += vec
        temp_vec /= len(habitat_embeddings[key])
        habitat_embeddings[key] = temp_vec

In [105]:
len(habitat_map_originals)

3602

In [106]:
habitat_embeddings

{'fermented cheese': array([-0.00124951,  0.09627202, -0.01634476,  0.06334709,  0.02650735,
        -0.06484498, -0.04169053,  0.04486798, -0.07212708,  0.05576644,
        -0.06307281,  0.08622901, -0.02984504, -0.13176938,  0.0700487 ,
         0.09546501, -0.02166744, -0.11599933,  0.00079243,  0.14958021,
         0.07443528, -0.03186995, -0.10722319,  0.04089891, -0.08690316,
        -0.01658884,  0.0449776 , -0.04056346,  0.09502719, -0.04826566,
        -0.04900088, -0.05026881, -0.28009997,  0.0045067 ,  0.02197481,
        -0.00614672, -0.08998885, -0.03816161, -0.02358121, -0.05575434,
         0.05139458,  0.10580041,  0.07917946, -0.10973523,  0.0417779 ,
        -0.04210231, -0.02931119,  0.09762399,  0.13456034, -0.0527993 ,
        -0.1806532 ,  0.0189025 ,  0.01489776, -0.08882207,  0.00314774,
         0.02877654, -0.01505821,  0.06293634, -0.08807333,  0.04688586,
        -0.10991332, -0.00546526,  0.0203589 , -0.01795142, -0.01786765,
        -0.02026355,  0.1012789

In [107]:
def predict_habitats_exact_matching(a1, txt, a1_name, processor, pred_file_name):
    pred_file = open(pred_file_name, "w")
    cands = 0
    matches = 0
    match_list = {}
    a1 = a1.split("\n")
    count = 1
    for line in a1:
        found = False
        if "Habitat" in line:
            cands += 1
            name = (line.split("\t")[2])
            named_entity = name
            if name != name.upper():
                name = name.lower()
                named_entity = processor(name)
            if named_entity in habitat_map:
                matches += 1
                match = name + " - " + habitat_map[named_entity] + "---exact"
                match_list[line.split("\t")[0]] = [habitat_map[named_entity], named_entity, "exact"]
                found = True
            else:
                #print(name)
                pre_named_entity = get_headword(name)
                #print(pre_named_entity)
                named_entity = processor(pre_named_entity)
                if named_entity in habitat_map:
                    matches += 1
                    match = name + " - " + habitat_map[named_entity] + "---headwordexact"
                    print(match)
                    match_list[line.split("\t")[0]] = [habitat_map[named_entity], name, "headwordexact", named_entity]
                    found = True
        if found:
            pred_file.write("N" + str(count) +"\tOntoBiotope Annotation:" + line.split("\t")[0] + " Referent:OBT:" + match_list[line.split("\t")[0]][0] + "\n")
            count += 1
    #print(a1_name)
    #print("Out of", cands, "candidates,", matches, "matches found")
    #print()
    pred_file.close()
    return match_list

In [132]:
def predict_habitats(a1, txt, a1_name, processor, pred_file_name, model):
    pred_file = open(pred_file_name, "w")
    cands = 0
    matches = 0
    match_list = {}
    a1 = a1.split("\n")
    count = 1
    for line in a1:
        found = False
        if "Habitat" in line:
            name = (line.split("\t")[2])
            named_entity = name
            if name != name.upper():
                name = name.lower()
                named_entity = processor(name)
            if named_entity in habitat_map:
                matches += 1
                match = name + " - " + habitat_map[named_entity] + "---exact"
                match_list[line.split("\t")[0]] = [habitat_map[named_entity], named_entity, "exact"]
                pred_file.write("N" + str(count) +"\tOntoBiotope Annotation:" + line.split("\t")[0] + " Referent:OBT:" + match_list[line.split("\t")[0]][0] + "\n")
                count += 1
                continue
            #print(name)
            pre_named_entity = get_headword(name)
            #print(pre_named_entity)
            named_entity = processor(pre_named_entity)
            if named_entity in habitat_map:
                matches += 1
                match = name + " - " + habitat_map[named_entity] + "---headwordexact"
                #print(match)
                match_list[line.split("\t")[0]] = [habitat_map[named_entity], name, "headwordexact", named_entity]
                found = True
                pred_file.write("N" + str(count) +"\tOntoBiotope Annotation:" + line.split("\t")[0] + " Referent:OBT:" + match_list[line.split("\t")[0]][0] + "\n")
                count += 1
                continue
            """
            cands += 1
            name = (line.split("\t")[2])
            name = name.replace("-", " ")
            named_entity = name
            entity_vec = get_phrase_embedding(named_entity, model, processor)
            match_id = ""
            if type(entity_vec) != int:
                largest_prod = np.dot(entity_vec, habitat_embeddings[list(habitat_embeddings)[0]])/(norm(entity_vec)*norm(habitat_embeddings[list(habitat_embeddings)[0]]))
                match_id = habitat_map[list(habitat_embeddings)[0]]
                for habitat in habitat_embeddings:
                    prod = np.dot(entity_vec,habitat_embeddings[habitat])/(norm(entity_vec)*norm(habitat_embeddings[habitat]))
                    if prod > largest_prod:
                        largest_prod = prod
                        match_id = habitat_map[habitat]
                
                match_list[line.split("\t")[0]] = [match_id, named_entity, "w2v"]
            
            pred_file.write("N" + str(count) +"\tOntoBiotope Annotation:" + line.split("\t")[0] + " Referent:OBT:" + match_id + "\n")
            count += 1
            """
    #print(a1_name)
    #print("Out of", cands, "candidates,", matches, "matches found")
    #print()
    pred_file.close()
    return match_list

In [133]:
annotations_test = {}
for i in range(len(BB_dev_txt)):
    a1 = open(dev_path + BB_dev_a1[i],encoding="utf8").read()
    txt = open(dev_path + BB_dev_txt[i],encoding="utf8").read()
    pred_file_name = "./dev_preds/" + BB_dev_a2[i]
    annotations_test[BB_dev_a1[i]] = predict_habitats(a1, txt, BB_dev_a1[i], phrase_reducer, pred_file_name, model)

In [134]:
annotations_test

{'BB-norm-10496597.a1': {'T3': ['001792', 'gastric', 'exact'],
  'T4': ['001577', 'gastric mucosa', 'exact'],
  'T5': ['000196',
   'gastric mucosal-associated lymphoid tissue',
   'headwordexact',
   'tissue'],
  'T7': ['001792', 'gastric', 'exact'],
  'T9': ['000061', 'monoclonal b cells', 'headwordexact', 'cell'],
  'T11': ['001792', 'gastric', 'exact'],
  'T12': ['003215', 'gastric malt', 'headwordexact', 'malt'],
  'T13': ['003220',
   'patients with atypical lymphoid infiltrates',
   'headwordexact',
   'patient'],
  'T14': ['003220',
   'patients with low-grade malt lymphoma',
   'headwordexact',
   'patient'],
  'T15': ['003220',
   'patients with helicobacter pylori-chronic active gastritis',
   'headwordexact',
   'patient'],
  'T16': ['003220',
   'patients with high-grade primary gastric lymphoma',
   'headwordexact',
   'patient'],
  'T17': ['001792', 'gastric', 'exact'],
  'T23': ['003220',
   'patients with chronic active gastritis',
   'headwordexact',
   'patient'],
  

In [135]:
def test_habitats(a1, a2, processor):
    
    a2 = a2.split("\n")
    a1 = a1.split("\n")
    match_list = {}
    for annotation in a2:
        a1_line = 0
        t = ""
        oid = ""
        #print(annotation)
        if "OntoBiotope" in annotation:
            t = re.findall(r"(?<=Annotation:)T[0-9]+", annotation)[0]
            oid = re.findall(r"(?<=Referent:OBT:)[0-9]+", annotation)[0]
            named_entity = ""
            while t not in a1[a1_line] and a1_line < len(a1)-1:
                a1_line += 1
            if "Habitat" in a1[a1_line]:
                named_entity = (a1[a1_line].split("\t")[2])
                if named_entity != named_entity.upper():
                    named_entity = named_entity.lower()
                    named_entity = processor(named_entity)
                match_list[t] = [oid, named_entity]
    return match_list

In [136]:
dev_a2 = {}
for i in range(len(BB_dev_txt)):
    a1 = open(dev_path + BB_dev_a1[i],encoding="utf8").read()
    a2 = open(dev_path + BB_dev_a2[i],encoding="utf8").read()
    dev_a2[BB_dev_a1[i]] = test_habitats(a1,a2, phrase_reducer)

In [137]:
true_pred_count = 0
false_pred_count = 0
total_count = 0
for key in dev_a2:
    """print()
    print(key)
    print(dev_a2[key])
    print("**************")"""
    total_count += len(dev_a2[key])
    if key in annotations_test:
        for pred in annotations_test[key]:            
            if annotations_test[key][pred][0] == dev_a2[key][pred][0]:
                true_pred_count += 1
                #print(key)
                #print("Prediction:", annotations_test[key][pred]," Real:", dev_a2[key][pred])
                #print("Prediction:", habitat_map_originals[annotations_test[key][pred][0]], " Real:", habitat_map_originals[dev_a2[key][pred][0]])
            else:
                print(key)
                print("Prediction:", annotations_test[key][pred]," Real:", dev_a2[key][pred])
                print("Prediction:", habitat_map_originals[annotations_test[key][pred][0]], " Real:", habitat_map_originals[dev_a2[key][pred][0]])
                false_pred_count += 1

BB-norm-10496597.a1
Prediction: ['000196', 'gastric mucosal-associated lymphoid tissue', 'headwordexact', 'tissue']  Real: ['000334', 'gastric mucosal-associated lymphoid tissue']
Prediction: animal tissue  Real: lymphatic system part
BB-norm-10496597.a1
Prediction: ['000061', 'monoclonal b cells', 'headwordexact', 'cell']  Real: ['001623', 'monoclonal b cell']
Prediction: cell  Real: lymphocyte
BB-norm-10496597.a1
Prediction: ['003215', 'gastric malt', 'headwordexact', 'malt']  Real: ['000334', 'gastric malt']
Prediction: malt  Real: lymphatic system part
BB-norm-10496597.a1
Prediction: ['003220', 'patients with helicobacter pylori-chronic active gastritis', 'headwordexact', 'patient']  Real: ['003269', 'patient with helicobacter pylori-chronic active gastritis']
Prediction: patient  Real: patient with infectious disease
BB-norm-10496597.a1
Prediction: ['000061', 'cd20-positive cells', 'headwordexact', 'cell']  Real: ['001623', 'cd20-positive cell']
Prediction: cell  Real: lymphocyte


Prediction: cheese  Real: scimudin
BB-norm-F-20167385-003.a1
Prediction: ['001480', 'casera cheese', 'headwordexact', 'cheese']  Real: ['003483', 'casera cheese']
Prediction: cheese  Real: valtellina casera
BB-norm-F-20167385-004.a1
Prediction: ['002697', 'community of staphylococcus and micrococci-corynebacteria groups', 'headwordexact', 'community']  Real: ['000113', 'community of staphylococcus and micrococci-corynebacteria group']
Prediction: community  Real: microflora
BB-norm-F-20167385-004.a1
Prediction: ['001480', 'casera cheeses', 'headwordexact', 'cheese']  Real: ['003483', 'casera cheese']
Prediction: cheese  Real: valtellina casera
BB-norm-F-20167385-004.a1
Prediction: ['001480', 'taleggio cheeses', 'headwordexact', 'cheese']  Real: ['003525', 'taleggio cheese']
Prediction: cheese  Real: taleggio
BB-norm-F-20167385-004.a1
Prediction: ['001480', 'gorgonzola cheeses', 'headwordexact', 'cheese']  Real: ['003504', 'gorgonzola cheese']
Prediction: cheese  Real: gorgonzola
BB-nor

In [138]:
recall = true_pred_count/total_count
recall

0.49508196721311476

In [139]:
precision = true_pred_count/(true_pred_count + false_pred_count)
precision

0.668141592920354

## Denemeler